In [1]:
from sklearn.base import TransformerMixin
from sklearn.preprocessing import Imputer
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn
import matplotlib
import time
import json

%matplotlib inline

In [2]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [3]:
df_courses = pd.read_json('/Users/vml/estagiario_pd/datasets/courses.json', orient='records')
df_sessions = pd.read_json('/Users/vml/estagiario_pd/datasets/sessions.json', orient='records')
df_sfs = pd.read_json('/Users/vml/estagiario_pd/datasets/student_follow_subject.json', orient='records')
df_students = pd.read_json('/Users/vml/estagiario_pd/datasets/students.json', orient='records')
df_subjects = pd.read_json('/Users/vml/estagiario_pd/datasets/subjects.json', orient='records')
df_subscriptions = pd.read_json('/Users/vml/estagiario_pd/datasets/subscriptions.json', orient='records')
df_universities = pd.read_json('/Users/vml/estagiario_pd/datasets/universities.json', orient='records')

In [4]:
df_students = df_students[['Id', 'CourseId', 'UniversityId', 'State', 'City', 'RegisteredDate', 'StudentClient', 'SignupSource']]
df_sfs = df_sfs[['StudentId', 'SubjectId', 'FollowDate']]
df_subscriptions = df_subscriptions[['StudentId', 'PlanType', 'PaymentDate']]
df_sessions = df_sessions[['StudentId', 'SessionStartTime', 'StudentClient']]

In [5]:
def create_id_to_name():
    for name in ("subjects","courses","universities"):
        LOC = f"""
global change_id_to_name_{name}
change_id_to_name_{name} = df_{name}.set_index('Id').to_dict()['Name']
"""
        exec(LOC)

def replace_all_id_to_name():
    for name in ("subjects","courses","universities"):
        if name == "subjects":
            df_sfs.SubjectId.replace(change_id_to_name_subjects, inplace = True)
        elif name == "courses":
            df_students.CourseId.replace(change_id_to_name_courses, inplace = True)
        elif name == "universities":
            df_students.UniversityId.replace(change_id_to_name_universities,inplace = True)

def time_tuple_conversion(df):
    # para análise de séries temporais
    for date in ("RegisteredDate", "PaymentDate", "FollowDate", "SessionStartTime"):
        try:
            if date in df.columns:
                df[date] = df[date].apply(lambda x: x[:19]).apply(time.strptime, args=["%Y-%m-%d %H:%M:%S"]).apply(lambda x: x[:6])
        except:
            print(f"Provavelmente já foi convertida a coluna {date}")
            pass
            
            
create_id_to_name()
replace_all_id_to_name()

In [ ]:
# explicar aqui como usei o simple imputer alterado pra fazer imputacao de 

dados_pra_imputar_1 = df_students[["CourseId","UniversityId","State"]]
dados_pra_imputar_2 = df_students[["CourseId","UniversityId","City"]]

df_students[["City","State","UniversityId"]].groupby(["UniversityId"]).count().sort_values(by=["City", "State"], ascending = False).head(150)
quantos_uni = df_students[["City","State","UniversityId"]].groupby(["UniversityId"]).count().sort_values(by=["City", "State"], ascending = False).head(150).index.values
top200_universidades = df_students.UniversityId.value_counts().head(200)
top200_universidades = tuple(top200_universidades.index.values)

def imputando_dados_faltantes():
    for nome in top200_universidades:
        try:
            dados_pra_imputar_1[dados_pra_imputar_1["UniversityId"]==nome] = DataFrameImputer().fit_transform(dados_pra_imputar_1[dados_pra_imputar_1["UniversityId"]==nome])
            dados_pra_imputar_2[dados_pra_imputar_2["UniversityId"]==nome] = DataFrameImputer().fit_transform(dados_pra_imputar_2[dados_pra_imputar_2["UniversityId"]==nome])
        except:
            print("nao foi")
            pass
        
imputando_dados_faltantes()
df_students[["CourseId", "UniversityId", "State"]] = dados_pra_imputar_1
df_students[["CourseId", "UniversityId", "City"]] = dados_pra_imputar_2

df_students.info()

In [15]:
# pode ser conveniente montar um classificador de palavras aleatorias em portugues,
# pra avaliar ddaddos no BD que são irrelevantes ou desconsiderar, ou contar quantas pessoas usam essas palavras
# avaliar se sao mesmo aleatoria

In [5]:
# descrever no jupyter todos esses databases montados, o por que de cada um e por que separados, o por que ordenei as colunas de cada forma, como pode ajudar a visualizar os dados;
# também ver quais colunas tirar no final da análise

students_subjects = df_sfs.rename(columns = {"StudentId":"Id"}).merge(df_students, on= ["Id"]).rename(
    columns = {"Id":"StudentId", "SubjectId":"SubjectName", "CourseId": "CourseName",
               "UniversityId":"UniversityName" })[
['StudentId', 'CourseName', 'UniversityName', 'RegisteredDate', 'State', 'City', 'SubjectName', 'FollowDate', 'StudentClient', 'SignupSource']];

students_subjects_subscriptions = df_sfs.rename(columns = {"StudentId":"Id"}).merge(df_students, on= ["Id"]).rename(
    columns = {"Id":"StudentId", "SubjectId":"SubjectName", "CourseId": "CourseName",
               "UniversityId":"UniversityName" })[
['StudentId', 'CourseName', 'UniversityName', 'RegisteredDate', 'State', 'City', 'SubjectName', 'FollowDate', 'StudentClient', 'SignupSource']].merge(df_subscriptions, on = ["StudentId"]);

students_sessions = df_sessions.rename(columns = {"StudentId":"Id"}).merge(df_students, on = ["Id"]).rename(
    columns = {"Id":"StudentId","CourseId":"CourseName","UniversityId":"UniversityName"})[['StudentId','CourseName','UniversityName','SessionStartTime','StudentClient_x','StudentClient_y','State','City','RegisteredDate','SignupSource']];

sessions_subscriptions = df_sessions.merge(df_subscriptions, on= ["StudentId"])[['StudentId','SessionStartTime','StudentClient','PaymentDate', 'PlanType']];

students_subscriptions = df_subscriptions.rename(columns = {"StudentId":"Id"}).merge(df_students, on= ["Id"]).rename(
    columns = {"Id":"StudentId", "CourseId": "CourseName",
               "UniversityId":"UniversityName" })[['StudentId', 'UniversityName', 'CourseName', 'PlanType', 'PaymentDate', 'State', 'City', 'RegisteredDate', 'SignupSource', 'StudentClient']];

In [6]:
# tá demorando bem eim...
for database in (students_subjects, students_subjects_subscriptions, students_sessions, sessions_subscriptions, students_subscriptions):
    time_tuple_conversion(database)

In [6]:
df_students.SignupSource.value_counts() # da pra fazer um histograma aqui

Facebook    30346
Google      19312
Email       10342
Name: SignupSource, dtype: int64

In [7]:
df_students.StudentClient.value_counts().head(7)

Website                     38415
Webapp                       3214
Android | 6.0.1 | sdk 23     2137
Android | 7.0 | sdk 24       1059
Android | 6.0 | sdk 23       1045
Android | 5.1.1 | sdk 22      646
Android | 4.4.4 | sdk 19      434
Name: StudentClient, dtype: int64

In [8]:
df_students.UniversityId.value_counts().head(5)

ESTÁCIO        9030
UNIP           4426
UNOPAR         1857
UNINTER        1856
ESTÁCIO EAD    1478
Name: UniversityId, dtype: int64

In [9]:
df_students.CourseId.value_counts().head(5)

Direito                               9916
Administração                         4168
Engenharia Civil                      3562
Contabilidade / Ciências Contábeis    2714
Pedagogia                             2566
Name: CourseId, dtype: int64

In [10]:
df_subscriptions.head(10)

,StudentId,PlanType,PaymentDate
0,29037b0a52c5b576d32ac5842fd5b7e5991c5bef93150f...,Mensal,2017-11-14 19:52:36.756325
1,b2bace77d15c3dfaf57bf9916f0e9924d12f89bdc0624c...,Mensal,2017-11-08 11:52:09.830538
2,f423d6fe2f8964db634c7806e03b0f6567e992e994f06f...,Mensal,2017-11-05 21:27:47.793878
3,55ccbe518d2edbbd527eddca1431c7f7f3a4676bdbee38...,Mensal,2017-11-15 14:36:42.348867
4,b1b0f63fe3e4820cb082531af7c8911cd380e5ec8fe822...,Mensal,2017-11-12 22:19:54.575387
5,ed46832f6b716fb2e2f6f11b43965f2e0a3f2f8f8994ce...,Mensal,2017-11-22 01:03:09.059587
6,69b7bee32821cf76b9ffd5deffca1a01c95499cea24a45...,Mensal,2017-11-24 19:03:49.902349
7,6553923125fe6364e16aacf5656426809bb134b005f182...,Mensal,2017-11-11 21:01:59.149344
8,3903a334d1af8ce838bc2b7f497f11c40437573c1ad30b...,Mensal,2017-11-12 16:41:19.995949
9,4487f81a4ea9b3c3cf58aa31ecc21c30d6fb186993af61...,Mensal,2017-11-21 11:52:55.082745


In [11]:
list_of_datetimes_register = list(df_students.RegisteredDate.values);

In [12]:
print(df_students.City.value_counts().head(10),"\n")
print(df_students.State.value_counts().head(10))

Rio de Janeiro    2415
São Paulo         1089
Salvador           667
Belo Horizonte     552
Fortaleza          488
Curitiba           433
Recife             431
Goiânia            396
Brasília           287
São Luís           280
Name: City, dtype: int64 

Rio de Janeiro       5168
São Paulo            3870
Minas Gerais         2417
Paraná               1438
Bahia                1354
Ceará                1236
Rio Grande do Sul    1192
Pernambuco            954
Santa Catarina        876
Goiás                 815
Name: State, dtype: int64


In [13]:
for i in list_of_datetimes_register:
    list_of_datetimes_register[list_of_datetimes_register.index(i)] = i[:19]

In [ ]:
dates = matplotlib.dates.date2num(list_of_datetimes_register);
matplotlib.pyplot.plot_date(dates, values);

In [14]:
df_sfs.SubjectId.value_counts().head(5)

Bioquímica                4223
Cálculo I                 4053
Anatomia Humana           3950
Direito Constitucional    3926
Matemática Financeira     3405
Name: SubjectId, dtype: int64

In [413]:
df_students.UniversityId.value_counts()

ESTÁCIO                 9030
UNIP                    4426
UNOPAR                  1857
UNINTER                 1856
ESTÁCIO EAD             1478
Anhanguera               674
UNIDERP - ANHANGUERA     622
PITÁGORAS                575
USP-SP                   550
UFRJ                     526
UNINOVE                  503
UNB                      477
UNIVERSO                 430
UFMG                     406
PUC-MINAS                399
UFF                      368
UFPE                     362
UNESP                    338
UFBA                     307
ANHANGUERA               301
UVA                      298
UNIASSELVI               298
UFC                      294
UTFPR                    292
UFPR                     284
UFSC                     277
ULBRA                    274
UFRGS                    270
UFPI                     250
FMU                      249
                        ... 
PORTAL                     1
IST                        1
FADFESPSP                  1
GRANTIETE     

In [6]:
df_students[["City","State","UniversityId"]].groupby(["UniversityId"]).count().sort_values(by=["City", "State"], ascending = False).head(150)
quantos_uni = df_students[["City","State","UniversityId"]].groupby(["UniversityId"]).count().sort_values(by=["City", "State"], ascending = False).head(150).index.values
top200_universidades = df_students.UniversityId.value_counts().head(200)
top200_universidades = tuple(top200_universidades.index.values)

In [24]:
top500_universidades = df_students.UniversityId.value_counts().head(500)
top500_universidades = tuple(top500_universidades.index.values)

In [26]:
top500_universidades = tuple(top500_universidades.index.values)

In [ ]:
top